In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_preparation import *
from interpolation import *
# Lin interpolation
from scipy.interpolate import griddata
import seaborn as sns
# Kriging
from pykrige.uk import UniversalKriging
import skgstat as skg

In [2]:
from locale import normalize
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import argparse
from interpolation import *
from data_preparation import *
from trainNN import *

def build_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

def path_for_plot(sampling_distance_y, sampling_distance_x):
    path = f'plots/NonRandomResapling_x{sampling_distance_x}_y{sampling_distance_y}/'
    build_path(path)
    return path






    

2022-05-17 09:26:37.007548: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-17 09:26:37.007609: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [98]:
length = 700
sampling_distance_x = 4
sampling_distance_y = 24
verbose = False
epochs = 1500


In [99]:
        whole_map = pd.read_csv('WholeMap_Rounds_40_to_17.csv')
        map = stack_map(whole_map) # create a stacked map dataframe with columns x, y, z
        map = cut_map_len(map,length) # cut the map to the length of the map
        data_i, data_o = resample(map, sampling_distance_x, sampling_distance_y) # resample the map


In [206]:

def randomsampling(map,min_x, max_x, min_y, max_y, sampling_distance_x, sampling_distance_y):
        
        
        
        amount_x_vals = len([x for x in range(int(sampling_distance_x/2),max_x+1,sampling_distance_x)])
       
        amount_y_vals = len([x for x in range(int(sampling_distance_y/2),max_y+1,sampling_distance_y)])
       
        numrows = amount_x_vals * amount_y_vals
        
        x_vals = np.random.randint(min_x, max_x,int(amount_x_vals))

        y_vals = np.random.randint(min_y, max_y,int(amount_y_vals))
        
        
        sample_map = map.copy().query("x in @x_vals")
        sample_map = sample_map.query("y in @y_vals")
       
        while len(sample_map) < numrows:
                
                x_vals = np.random.randint(min_x, max_x,int(1))

                y_vals = np.random.randint(min_y, max_y,int(1))
        
                next_sample_map = map.copy().query("x in @x_vals")
                next_sample_map = next_sample_map.query("y in @y_vals")
                
                sample_map = pd.concat([sample_map,next_sample_map]).drop_duplicates(keep='first')
        
        if len(sample_map) >= numrows:
                randomsampling(map,min_x, max_x, min_y, max_y, sampling_distance_x, sampling_distance_y)
                
       
        return sample_map

In [207]:
randomsampling(map,0,126,12,8366,4,24)

KeyboardInterrupt: 

In [208]:
amount_x_vals = len([x for x in range(int(sampling_distance_x/2),max_x+1,sampling_distance_x)])
       
amount_y_vals = len([x for x in range(int(sampling_distance_y/2),max_y+1,sampling_distance_y)])
       
numrows = amount_x_vals * amount_y_vals

 

In [209]:
numrows

11200

In [223]:

       x_vals = np.random.randint(min_x, max_x,int(amount_x_vals))

       y_vals = np.random.randint(min_y, max_y,int(amount_y_vals))
       
                

In [224]:
len(y_vals)

350

In [225]:
sample_map = map.copy().query("x in @x_vals")
sample_map = sample_map.query("y in @y_vals")

In [226]:
while len(sample_map) < numrows:
                print(len(sample_map))
                
                x_vals = np.random.randint(min_x, max_x,1)

                y_vals = np.random.randint(min_y, max_y,1)
        
                next_sample_map = map.copy().query("x in @x_vals")
                next_sample_map = next_sample_map.query("y in @y_vals")
                
                sample_map = pd.concat([sample_map,next_sample_map]).drop_duplicates(keep='first')

9860
9861
9862
9863
9864
9865
9866
9867
9868
9869
9870
9871
9872
9873
9874
9875
9876
9877
9878
9879
9880
9881
9882
9883
9884
9885
9886
9887
9888
9889
9890
9891
9892
9893
9894
9895
9896
9897
9898
9899
9900
9901
9902
9903
9904
9904
9905
9906
9907
9908
9909
9910
9911
9912
9913
9914
9915
9916
9917
9918
9919
9920
9921
9922
9923
9924
9925
9926
9927
9928
9929
9930
9931
9932
9933
9934
9935
9936
9937
9938
9939
9940
9941
9942
9943
9944
9945
9946
9947
9948
9949
9950
9951
9952
9953
9954
9955
9956
9957
9958
9959
9960
9961
9962
9963
9964
9965
9966
9967
9968
9969
9970
9971
9972
9973
9974
9975
9976
9977
9978
9979
9980
9981
9982
9983
9984
9985
9986
9987
9988
9989
9990
9991
9992
9993
9994
9995
9996
9997
9998
9999
10000
10001
10002
10003
10004
10005
10006
10007
10008
10009
10010
10011
10012
10013
10014
10015
10016
10017
10018
10019
10020
10021
10022
10023
10024
10025
10026
10027
10028
10029
10030
10031
10032
10033
10034
10035
10036
10037
10038
10039
10040
10041
10042
10043
10044
10045
10046
10047
10048
1

In [227]:
sample_map

,x,y,z
8985,15,69,-27.698152
8987,17,69,-30.607096
8995,25,69,-34.762665
8999,29,69,-29.183149
9005,35,69,-27.908019
...,...,...,...
574579,109,4419,-30.509416
642250,50,4940,-32.743565
761367,87,5856,-28.144048
552740,110,4251,-28.728806


In [23]:
kriging_interpol_stacked['z'] - 

,x,y,z
0,3.0,36.0,-26.828200
1,9.0,36.0,-27.497435
2,15.0,36.0,-28.261022
3,21.0,36.0,-29.058449
4,27.0,36.0,-29.853408
...,...,...,...
2431,99.0,8316.0,-22.586871
2432,105.0,8316.0,-23.159391
2433,111.0,8316.0,-22.276568
2434,117.0,8316.0,-22.115377


In [4]:

      
        # Lin interpolation
        
        lin_interpol, lin_interpol_stacked = grid_interpolation(data_i, data_o)

        # Kriging interpolation

        kriging_interpol, kriging_interpol_stacked = kriging_skg(data_i, data_o)
        
        # NN prediction
        x_train = data_i[['x', 'y']]
        y_train = data_i[['z']]

        x_val = data_o[['x', 'y']]
        y_val = data_o[['z']]

        model,minval_fornom =  train(x_train, y_train, x_val, y_val,length,build_model(x_train),epochs,sampling_distance_x, sampling_distance_y, verbose=verbose, normalize=normalize)
        
        predictions, pred_stacked = prediction(model,data_o, x_val, sampling_distance_x, sampling_distance_y) 
        

2022-05-16 12:07:50.641989: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-16 12:07:50.642015: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-16 12:07:50.642035: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (schmijul-thinkpad): /proc/driver/nvidia/version does not exist
2022-05-16 12:07:50.642309: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
ground_truth = data_o.pivot_table(index='y', columns='x', values='z')

In [18]:
np.mean(np.mean((predictions-ground_truth)**2))

/home/schmijul/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


9.406135021584955

In [22]:
pred_stacked.index = data_o.index

In [31]:
np.mean((kriging_interpol_stacked['z']-data_o['z'])**2)

3.521177713903579

In [30]:
kriging_interpol_stacked.index = data_o.index

In [6]:
f= open(f"test.txt","a")
f.write('test')
f.close()